# Test Redshift Database

## Imports

In [ ]:
import os
import glob
from pytz import utc
import numpy as np
from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from desiutil.iers import freeze_iers
from desiutil.log import get_logger, DEBUG, INFO
from desispec.io.meta import faflavor2program
from desispec.io.util import checkgzip
import desispec.database.redshift as dsr
freeze_iers()

## Important Setup

In [ ]:
specprod = os.environ['SPECPROD'] = 'fuji'
overwrite = False

## Consistency checks on exposures, frames and tiles

### Load two versions of tiles file

In [ ]:
tiles_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.fits'), hdu='TILE_COMPLETENESS')

In [ ]:
tiles_fits

In [ ]:
tiles_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'), format='ascii.csv')

In [ ]:
tiles_csv

### Are the two tiles files self-consistent?

In [ ]:
for row in range(len(tiles_fits)):
    for col in tiles_fits.colnames:
        try:
            assert tiles_fits[row][col] == tiles_csv[row][col]
        except AssertionError:
            print(row, col)

### Load two versions of exposures file

In [ ]:
exposures_fits = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='EXPOSURES')
frames = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'), hdu='FRAMES')

In [ ]:
exposures_fits

In [ ]:
exposures_csv = Table.read(os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.csv'), format='ascii.csv')

In [ ]:
exposures_csv

### Are the two exposures files self-consistent?

In [ ]:
for row in range(len(exposures_fits)):
    for col in exposures_fits.colnames:
        try:
            assert exposures_fits[row][col] == exposures_csv[row][col]
        except AssertionError:
            try:
                assert np.around(exposures_fits[row][col].astype(float), 1) == exposures_csv[row][col]
            except AssertionError:
                try:
                    assert np.around(exposures_fits[row][col].astype(float), 2) == exposures_csv[row][col]
                except AssertionError:
                    try:
                        assert np.around(exposures_fits[row][col].astype(float), 3) == exposures_csv[row][col]
                    except AssertionError:
                        print(row, col, exposures_fits[row][col], exposures_csv[row][col])

### What programs are present?

In [ ]:
np.unique(faflavor2program(exposures_fits['FAFLAVOR']))

In [ ]:
np.unique(exposures_fits['GOALTYPE'])

In [ ]:
np.unique(faflavor2program(tiles_fits['FAFLAVOR']))

In [ ]:
program = faflavor2program(exposures_fits['FAFLAVOR'])
assert (exposures_fits['PROGRAM'] == program).all()



In [ ]:
for survey in np.unique(exposures_fits['SURVEY']):
    print(f"'{survey}': ", np.unique(program[exposures_fits['SURVEY'] == survey]).tolist(), ',', sep='')

### Compare frames to exposures

In [ ]:
for expid in frames['EXPID']:
    assert expid in exposures_fits['EXPID']

In [ ]:
for k, expid in enumerate(exposures_fits['EXPID']):
    assert (frames['NIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['NIGHT']).all()
    assert (frames['TILEID'][frames['EXPID'] == expid] == exposures_fits[k]['TILEID']).all()
    assert (frames['TILERA'][frames['EXPID'] == expid] == exposures_fits[k]['TILERA']).all()
    assert (frames['TILEDEC'][frames['EXPID'] == expid] == exposures_fits[k]['TILEDEC']).all()
    # assert (frames['MJD'][frames['EXPID'] == expid] == exposures_fits[k]['MJD']).all()
    # assert (frames['EXPTIME'][frames['EXPID'] == expid] == exposures_fits[k]['EXPTIME']).all()
    assert (frames['AIRMASS'][frames['EXPID'] == expid] == exposures_fits[k]['AIRMASS']).all()
    # assert (frames['EBV'][frames['EXPID'] == expid] == exposures_fits[k]['EBV']).all()
    assert (frames['SEEING_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['SEEING_ETC']).all()
    assert (frames['EFFTIME_ETC'][frames['EXPID'] == expid] == exposures_fits[k]['EFFTIME_ETC']).all()
    # assert (frames['TSNR2_GPBDARK'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBDARK']).all()
    # assert (frames['TSNR2_ELG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_ELG']).all()
    # assert (frames['TSNR2_GPBBRIGHT'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBRIGHT']).all()
    # assert (frames['TSNR2_LYA'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LYA']).all()
    # assert (frames['TSNR2_BGS'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_BGS']).all()
    # assert (frames['TSNR2_GPBBACKUP'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_GPBBACKUP']).all()
    # assert (frames['TSNR2_QSO'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_QSO']).all()
    # assert (frames['TSNR2_LRG'][frames['EXPID'] == expid] == exposures_fits[k]['TSNR2_LRG']).all()
    assert (frames['SURVEY'][frames['EXPID'] == expid] == exposures_fits[k]['SURVEY']).all()
    assert (frames['GOALTYPE'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTYPE']).all()
    assert (frames['FAPRGRM'][frames['EXPID'] == expid] == exposures_fits[k]['FAPRGRM']).all()
    assert (frames['FAFLAVOR'][frames['EXPID'] == expid] == exposures_fits[k]['FAFLAVOR']).all()
    assert (frames['MINTFRAC'][frames['EXPID'] == expid] == exposures_fits[k]['MINTFRAC']).all()
    assert (frames['GOALTIME'][frames['EXPID'] == expid] == exposures_fits[k]['GOALTIME']).all()

### Compare tiles to exposures

In [ ]:
for row in tiles:
    # SURVEY is known issue
    # GOALTIME is known issue
    # OBSSTATUS is not in exposures
    w = exposures['TILEID'] == row['TILEID']
    assert len(exposures[w]) == row['NEXP']
    for column in ('SURVEY', 'FAPRGRM', 'FAFLAVOR', 'GOALTYPE'):
        try:
            assert (exposures[w][column] == row[column]).all()
        except AssertionError:
            print(row['TILEID'], row[column])
            print(exposures[w][column])
    try:
        assert (np.round(exposures[w]['TILERA'], decimals=2) == row['TILERA']).all()
    except AssertionError:
        assert (np.round(exposures[w]['TILERA'], decimals=3) == row['TILERA']).all()
    try:
        assert (np.round(exposures[w]['TILEDEC'], decimals=2) == row['TILEDEC']).all()
    except AssertionError:
        try:
            assert (np.round(exposures[w]['TILEDEC'], decimals=3) == row['TILEDEC']).all()
        except AssertionError:
            assert (np.round(exposures[w]['TILEDEC'], decimals=4) == row['TILEDEC']).all()
            # print(row['TILEID'], row['TILEDEC'])
            # print(np.round(exposures[w]['TILEDEC'], decimals=3))
    for column in ('EXPTIME', 'EFFTIME_ETC', 'EFFTIME_SPEC', 'EFFTIME_GFA', 'LRG_EFFTIME_DARK', 'ELG_EFFTIME_DARK', 'BGS_EFFTIME_BRIGHT', 'LYA_EFFTIME_DARK'):
        try:
            assert np.allclose(np.round(exposures[w][column].sum(), decimals=1), row[column])
        except AssertionError:
            print(row['TILEID'], row[column])
            print(column, np.round(exposures[w][column].sum(), decimals=1))
    for column in ('GOALTIME', 'MINTFRAC'):
        try:
            assert np.allclose(np.round(exposures[w][column], decimals=1), row[column])
        except AssertionError:
            try:
                assert np.allclose(np.round(exposures[w][column], decimals=2), row[column])
            except AssertionError:
                print(row['TILEID'], row[column])
                print(column, np.round(exposures[w][column], decimals=1))
    assert exposures[w]['NIGHT'].max() == row['LASTNIGHT']

## Check fiberassign files

In [ ]:
fiberassign = Table.read(fiberassign_files[0], hdu='FIBERASSIGN')
potential = Table.read(fiberassign_files[0], hdu='POTENTIAL_ASSIGNMENTS')

In [ ]:
fiberassign

In [ ]:
potential

## Converting MJD to datetime

In [ ]:
Time(exposures['MJD'], format='mjd').to_value('datetime').tolist()[0]

In [ ]:
Time(exposures['MJD'], format='mjd').to_value('datetime').tolist()[0].replace(tzinfo=utc)

## Test load tiles, exposures, frames

In [ ]:
dsr.log = get_logger(DEBUG, timestamp=True)
postgresql = dsr.setup_db(overwrite=overwrite, hostname='nerscdb03.nersc.gov', username='desidev_admin', schema=specprod, verbose=True)

In [ ]:
loader = [{'filepaths': os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'tiles-{specprod}.csv'),
           'tcls': dsr.Tile,
           'insert': {'faflavor': ('program',)},
           'convert': {'program': faflavor2program},
           'q3c': 'tilera',
          },
          {'filepaths': os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'),
           'tcls': dsr.Exposure,
           'hdu': 'EXPOSURES',
           'insert': {'faflavor': ('program',), 'mjd': ('date_obs',)},
           'convert': {'program': faflavor2program, 'date_obs': lambda x: Time(x, format='mjd').to_value('datetime').replace(tzinfo=utc)},
           'q3c': 'tilera',
          },
          {'filepaths': os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod, f'exposures-{specprod}.fits'),
           'tcls': dsr.Frame,
           'hdu': 'FRAMES',
           'preload': dsr._frameid,
          }]


In [ ]:
for l in loader:
    dsr.load_file(**l)

## Test load fiberassign

In [ ]:
fiberassign_files = [checkgzip(os.path.join(os.environ['DESI_TARGET'], 'fiberassign', 'tiles', 'trunk', (f"{tileid[0]:06d}")[0:3], f"fiberassign-{tileid[0]:06d}.fits"))
                     for tileid in dsr.dbSession.query(dsr.Tile.tileid).order_by(dsr.Tile.tileid)]
fiberassign_files

In [ ]:
loader = [{'filepaths': fiberassign_files,
           'tcls': dsr.Fiberassign,
           'hdu': 'FIBERASSIGN',
           'preload': dsr._tileid,
           'q3c': 'target_ra',
          },
          {'filepaths': fiberassign_files,
           'tcls': dsr.Potential,
           'hdu': 'POTENTIAL_ASSIGNMENTS',
           'preload': dsr._tileid,
          }]

In [ ]:
for l in loader:
    dsr.load_file(**l)

In [ ]:
dsr.load_file(**(loader[1]))